In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Outline

1. Import libraries
2. Read the dataframe
3. Preprocess (missing data, categorical encoding, detecting outliers)
4. Choose features to be clustered
5. Simulate cluster number using dendrogram and kmeans
6. Fit and predict cluster numbers in the dataframe
7. Interpret the findings

# Import Libraries
Other than numpy and pandas (have been imported on the first cell).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Read the Dataframe

In [ ]:
df_orig = pd.read_csv('/kaggle/input/clustering-data-id-gender-income-spending/ClusteringHSS.csv')

df = df_orig.copy()

df.info()

In [ ]:
df

# Preprocessing
1. Encode categorical variable
2. Handle missing data
3. Detect outlier

In [ ]:
print(df.Gender_Code.value_counts(), df.Region.value_counts(), sep='\n')

In [ ]:
# 1

df['Gender_Code'] = df['Gender_Code'].replace({"Female":1,"Male":0})
df['Region']= df['Region'].replace({"Urban":1, "Rural":0})

df.info()

In [ ]:
# 2 fill missing data

df = df.fillna(df.mean().round(0))

df.info()

In [ ]:
# 3 detect outlier

pd.DataFrame.boxplot(df.iloc[:,[3,4]])

# Choosing Features to be Clustered

Perhaps we can just focus on Income and Spending as we can agree that Customer ID and Gender perhaps give less value to our analysis.

In [ ]:
df

In [ ]:
df = df.iloc[:,2:]
x = df.values

In [ ]:
x

# Simulate Cluster Number
1. Using Hierarchical Clustering
2. Using KMeans

In [ ]:
# 1

plt.figure(figsize=(30,10))
dendrogram(linkage(x, method='ward'))
plt.show()

In [ ]:
# 2

wscc = []
for i in range(1,11):
    kmeans = KMeans(i)
    kmeans.fit(x)
    wscc.append(kmeans.inertia_)

plt.figure(figsize=(20,10))
plt.plot(range(1,11), wscc)
plt.show()

Perhaps the most optimum number of clusters based on Hierarchical Clustering and KMeans is **Three Clusters**.

In [ ]:
hc = AgglomerativeClustering(3)
cluster = hc.fit_predict(x)
df['Cluster'] = cluster

df

In [ ]:
df['Cluster'] = df['Cluster'].apply(lambda x: x+1)

df

In [ ]:
for c in df:
    grid = sns.FacetGrid(df, col='Cluster')
    grid.map(plt.hist, c)